In [2]:
import pandas as pd

In [3]:
openai=pd.read_csv(r"OPENAI_LLM.csv")

In [4]:
openai.columns

Index(['risks description', 'risk_type', 'keywords', 'sentence tags'], dtype='object')

In [5]:
openai.risk_type.head()

0    operational, traffic, drought, political, cons...
1    Tolerance-related risks, safety risks, crash r...
2         financial, safety, structural, environmental
3    Safety, Catastrophic, Political and Legal, Inc...
4    Payment delays, project delays, transmission r...
Name: risk_type, dtype: object

In [ ]:
os.environ["HUGGINGFACE"]

In [ ]:
loader=TextLoader(r"prj_abstract.txt")
text=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs=text_splitter.split_documents(text)

In [ ]:
def template_insertion(words: str):
    return f"""help me to identify and extract organizations in the ' {words} '? And if there are stakeholder in sentence act as organization, please mark it out and specify the type . Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]. If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show organization, keyword and tags appeared in sentence separately. Attention, you can only predict the category in above given criteria and no more than 15 words.
"""
# among [governments, design institutes, investment banking, contractors, consultants, subcontractors, suppliers, media, political party, environmental groups]. If it is an individual, please specify the type among [project managers, architects, engineers, surveyors, planners, workers, general public, journalists, politicians, or researchers]
# result=llm(stk_template)
# prompt_risk=PromptTemplate(template=stk_template, input_variables=["sentence"])
# prompt_risk.format(sentence=words)
# chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_risk)
# chainres=chain.run(words)
# mask=chainres.split("\n")
# print(mask)
# res=llm(template_insertion(words))
# print(res)

In [ ]:
def risk_template(words: str):
    return f"""help me to identify and extract risks in the '{words}'? please mark it out and specify the type . Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]. If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show risks, type of risks, keyword and tags appeared in sentence separately. Attention, you can only predict the output in above given criteria and no more than 15 words.No need to specific mark out on every sentence

    Answer example: "
    risks: Construction cost overrun, Ship collision and grounding \n

    risk_type : Financial, Technical \n

    keywords : Risk Management \n

    tags : Environment Science, Environment Studies, Finance" \n
    Answer no more than 20 words.
"""


def risk_template_backwords(words: str):
    return f"""help me to identify and extract risks in the '{words}'? please mark it out and specify the type . Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]. If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show type of risk follows its risks, keyword and tags appeared in sentence separately. Attention, you can only predict the category in above given criteria and no more than 20 words. No need to specific mark out on every sentence.
"""

In [ ]:
test_example, template_example=stake.iloc[0:2, ], pd.DataFrame([], columns=["source", "org/stk", "keyword", "tags"])
logging.basicConfig(filename="multithreading.log", level=logging.INFO)

In [ ]:
def split_dataframe(df=stake, chunk_size=12)->list:
    chunk, length=[], df.size
    for val in range(0, length, chunk_size): chunk.append((" ".join(df.iloc[val: val+chunk_size].Abstract.to_list()), val))
    return chunk

In [ ]:
def stk_keyword_split(res: list[str]):
    """res receive value from original feedback, right after OpenAI API return the value and split with \n"""
    feedback=[val.split(": ")[1] for val in res if val!='']
    if len(feedback)>3: feedback=[", ".join([feedback[0]+feedback[1]]), *feedback[2:]]
    elif len(feedback)<3: feedback=[*feedback, *[None]*(3-(len(feedback)))]
    return feedback

def risk_keyword_split(res: str):
    res=res.split("\n")
    feedback=[val.split(": ")[1] for val in res if val!='']
    if len(feedback)<4: feedback=[*feedback, *([None]*(4-len(feedback)))]
    return feedback

llm=OpenAI(temperature=0)

In [ ]:
risk_df=pd.DataFrame([], columns=["risks description", "risk_type", "keywords", "sentence tags", "source"])

In [ ]:
risk_length, chunk_size=len(risk), 10
for times in range(2594, risk_length, chunk_size):
    long_phrase=risk.Abstract[times: times+chunk_size].to_list()
    words=risk_template(" ".join(long_phrase))
    try:
        response=llm(words)
        risk_df.loc[len(risk_df)]=risk_keyword_split(response)+[long_phrase]
    except Exception as e:
        print(f"the possible reason is {e}, be reminded, processed risk up to {times}")
        break

In [ ]:
# long_phrase=risk.Abstract[1534: 1544].to_list()  # 176-181, 181--190
# words=risk_template(" ".join(long_phrase))
# response=llm(words)
# risk_df.loc[len(risk_df)]=risk_keyword_split(response)
apprea=pd.DataFrame(data=[], columns=["Source"])

In [ ]:
for val in range(156, 177, 10):
    phrases=" ".join(risk.Abstract[val: val+10].to_list())
    apprea.loc[len(apprea)]=phrases


In [ ]:
def tagger(long_words: tuple):
    template=template_insertion(long_words[0])
    try:
        feedback=llm(template).split("\n")
        integration=stk_keyword_split(feedback)
    except Exception as e:
        logging.error(f"Error processing row {long_words[1]}: {e}")
        sys.exit("Warning")
    # print(feedback)
    return [words, *integration]

In [ ]:
batches=split_dataframe(stake)

In [ ]:
# results=[]
# with ThreadPoolExecutor() as executor:
#     futures=executor.map(tagger, batches)
#
#     for future in futures:
#         results+=future

In [ ]:
doc=[val[0] for val in batches]

In [ ]:
temp_doc=[" ".join(stake.Abstract[val:val+2].to_list()) for val in range(0, 10, 2)]
temp_doc

In [ ]:
loader=DirectoryLoader("D:/Code Working Area/Jupyter/knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects/ExcelData/docu/", glob="**/*.txt")
document=loader.load()

In [ ]:
text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts=text_splitter.split_documents(document)

In [ ]:
embeddings=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
dosearch=Chroma.from_documents(texts, embeddings)

In [ ]:
qa=VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=dosearch)

In [ ]:
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type
EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return openai.Embedding.create(input=text_or_tokens, model=model)

In [ ]:
text=words[0:100]
try:
    values=get_embedding(text)
except openai.InvalidRequestError as e:
    print(e)
# ["data"][0]["embedding"]